Step 0. Unzip enron1.zip into the current directory.

Step 1. Traverse the dataset and create a Pandas dataframe. This is already done for you and should run without any errors. You should recognize Pandas from task 1.

In [15]:
import pandas as pd
import os

def read_spam():
    category = 'spam'
    directory = './enron1/spam'
    return read_category(category, directory)

def read_ham():
    category = 'ham'
    directory = './enron1/ham'
    return read_category(category, directory)

def read_category(category, directory):
    emails = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        with open(os.path.join(directory, filename), 'r') as fp:
            try:
                content = fp.read()
                emails.append({'name': filename, 'content': content, 'category': category})
            except:
                print(f'skipped {filename}')
    return emails

ham = read_ham()
spam = read_spam()

df = pd.DataFrame.from_records(ham)
df = df._append(pd.DataFrame.from_records(spam))
print(df.head(1))

skipped 0754.2004-04-01.GP.spam.txt
skipped 1414.2004-06-24.GP.spam.txt
skipped 2042.2004-08-30.GP.spam.txt
skipped 2140.2004-09-13.GP.spam.txt
skipped 2248.2004-09-23.GP.spam.txt
skipped 2526.2004-10-17.GP.spam.txt
skipped 2649.2004-10-27.GP.spam.txt
skipped 2698.2004-10-31.GP.spam.txt
skipped 3304.2004-12-26.GP.spam.txt
skipped 3364.2005-01-01.GP.spam.txt
skipped 4142.2005-03-31.GP.spam.txt
skipped 4201.2005-04-05.GP.spam.txt
skipped 4350.2005-04-23.GP.spam.txt
skipped 4566.2005-05-24.GP.spam.txt
skipped 5105.2005-08-31.GP.spam.txt
                             name                                  content  \
0  0001.1999-12-10.farmer.ham.txt  Subject: christmas tree farm pictures\n   

  category  
0      ham  


Step 2. Data cleaning is a critical part of machine learning. You and I can recognize that 'Hello' and 'hello' are the same word but a machine does not know this a priori. Therefore, we can 'help' the machine by conducting such normalization steps for it. Write a function `preprocessor` that takes in a string and replaces all non alphabet characters with a space and then lowercases the result.

In [16]:
import re

def preprocessor(e):
    return re.sub('[^A-Za-z]', ' ', e).lower()

Step 3. We will now train the machine learning model. All the functions that you will need are imported for you. The instructions explain how the work and hint at which functions to use. You will likely need to refer to the scikit learn documentation to see how exactly to invoke the functions. It will be handy to keep that tab open.

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# The CountVectorizer converts a text sample into a vector (think of it as an array of floats).
# Each entry in the vector corresponds to a single word and the value is the number of times the word appeared.
# Instantiate a CountVectorizer. Make sure to include the preprocessor you previously wrote in the constructor.
vectorizer = CountVectorizer(preprocessor=preprocessor)

# Use train_test_split to split the dataset into a train dataset and a test dataset.
# The machine learning model learns from the train dataset.
# Then the trained model is tested on the test dataset to see if it actually learned anything.
# If it just memorized for example, then it would have a low accuracy on the test dataset and a high accuracy on the train dataset.
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['category'], test_size=0.33, random_state=22)

# Use the vectorizer to transform the existing dataset into a form in which the model can learn from.
# Remember that simple machine learning models operate on numbers, which the CountVectorizer conveniently helped us do.
X_train_trans = vectorizer.fit_transform(X_train)

# Use the LogisticRegression model to fit to the train dataset.
# You may remember y = mx + b and Linear Regression from high school. Here, we fitted a scatter plot to a line.
# Logistic Regression is another form of regression. 
# However, Logistic Regression helps us determine if a point should be in category A or B, which is a perfect fit.
model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train_trans, y_train)

# Validate that the model has learned something.
# Recall the model operates on vectors. First transform the test set using the vectorizer. 
# Then generate the predictions.
X_test_trans = vectorizer.transform(X_test)
y_predict = model.predict(X_test_trans)

# We now want to see how we have done. We will be using three functions.
# `accuracy_score` tells us how well we have done. 
# 90% means that every 9 of 10 entries from the test dataset were predicted accurately.
# The `confusion_matrix` is a 2x2 matrix that gives us more insight.
# The top left shows us how many ham emails were predicted to be ham (that's good!).
# The bottom right shows us how many spam emails were predicted to be spam (that's good!).
# The other two quadrants tell us the misclassifications.
# Finally, the `classification_report` gives us detailed statistics which you may have seen in a statistics class.
accuracy_result = accuracy_score(y_test, y_predict)
print(accuracy_result)

cnf_matrix = confusion_matrix(y_test, y_predict)
print(cnf_matrix)

classif_report = classification_report(y_test, y_predict)
print(classif_report)



(1702, 35037)
0.9747356051703878
[[1186   21]
 [  22  473]]
              precision    recall  f1-score   support

         ham       0.98      0.98      0.98      1207
        spam       0.96      0.96      0.96       495

    accuracy                           0.97      1702
   macro avg       0.97      0.97      0.97      1702
weighted avg       0.97      0.97      0.97      1702



Step 4.

In [60]:
# Let's see which features (aka columns) the vectorizer created. 
# They should be all the words that were contained in the training dataset.
features = vectorizer.get_feature_names_out()
print(features)

# You may be wondering what a machine learning model is tangibly. It is just a collection of numbers. 
# You can access these numbers known as "coefficients" from the coef_ property of the model
# We will be looking at coef_[0] which represents the importance of each feature.
# What does importance mean in this context?
# Some words are more important than others for the model.
# It's nothing personal, just that spam emails tend to contain some words more frequently.
# This indicates to the model that having that word would make a new email more likely to be spam.
importance = model.coef_[0]
print(importance)

# Iterate over importance and find the top 10 positive features with the largest magnitude.
# Similarly, find the top 10 negative features with the largest magnitude.
# Positive features correspond to spam. Negative features correspond to ham.
# You will see that `http` is the strongest feature that corresponds to spam emails. 
# It makes sense. Spam emails often want you to click on a link.
imp_index_list = sorted(range(len(importance)), key=lambda i:importance[i], reverse=True)[:10]
print(imp_index_list)
imp_index_list_re = sorted(range(len(importance)), key=lambda i:importance[i])[:10]

for i in imp_index_list:
    print(features[i], importance[i])

for i in imp_index_list_re:
    print(features[i], importance[i])


['aa' 'aaa' 'aabvmmq' ... 'zzn' 'zzo' 'zzso']
[-2.29402036e-01  8.65359895e-04  9.70295480e-03 ... -1.47695155e-07
  1.92039244e-02  5.97729794e-02]
[22084, 15135, 22751, 15739, 24951, 20535, 26532, 23274, 15075, 26531]
no 1.0610069413329335
here 0.7506453238684079
only 0.7343325907095919
http 0.7299605440182397
prices 0.725753507977547
message 0.7185959004973724
removed 0.6992661332449402
pain 0.6945364667957212
hello 0.64531612764901
remove 0.6255517855545933
attached -1.6822617998578528
enron -1.55405954091313
doc -1.30802934969014
thanks -1.3050954596883126
daren -1.2537660526797558
pictures -1.1659037959112992
neon -1.147740848499446
deal -1.111696054710551
hpl -1.1041285060753325
meter -1.0473506613696473


Submission
1. Upload the jupyter notebook to Forage.

All Done!